# **Fruit Object Detection**
### - Bu çalışmada PyTorch kütüphanesi kullanılarak meyve tespiti(fruit detection) eğitimi ve testi yapılmıştır.
### - Model olarak TorchVision üzerinden Faster-RCNN Object Detector kullanılmıştır.


## Veri Özellikleri:
### - Fruit Images veriseti kullanılmıştır.
### - Elma, Muz ve portakal olmak üzere üç farklı sınıf bulunmaktadır.
### - 240 adet eğitim seti görseli ve etiketi, 60 adet test seti görseli ve etiketi içermektedir.
### - Etiketler "labelImg" ile etiketlenmiştir.
### - Etiketler XML formatındadır. Object detection şeklinde etiketlendiği içinde XML dosyaları objenin resimdeki koordinatlarını içermektedir.

## Öncelikle ortamda hazır bulunmayan, kullanmamız gereken bazı scriptleri yüklememiz gerekmektedir.

In [ ]:
!pip install pycocotools --quiet
!git clone https://github.com/pytorch/vision.git
!git checkout v0.3.0

!cp vision/references/detection/utils.py ./
!cp vision/references/detection/transforms.py ./
!cp vision/references/detection/coco_eval.py ./
!cp vision/references/detection/engine.py ./
!cp vision/references/detection/coco_utils.py ./

## Gerekli kütüphaneler çağırılır.

In [ ]:
import os
import random
import numpy as np
import pandas as pd
# for ignoring warnings
import warnings
warnings.filterwarnings('ignore')

import cv2

# XML ETIKET PARSE İÇİN
from xml.etree import ElementTree as et

# GÖRSELLEŞTİRME
import matplotlib.pyplot as plt
import matplotlib.patches as patches

# torchvision libraries
import torch
import torchvision
from torchvision import transforms as torchtrans  
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

# Eğitim için
from engine import train_one_epoch, evaluate
import utils
import transforms as T

# Veri Çoğaltma
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2

# Veri DosyaYapısı
### **--"train_zip"**
### -----"train"
### --------"apple_1.jpg"
### --------"apple_1.xml"
### --------"apple_n.jpg"
### --------"apple_n.xml"
### **--"test_zip"**
### -----"test"
### --------"apple_78.jpg"
### --------"apple_78.xml"
### --------"apple_n.jpg"
### --------"apple_n.xml"

## - Öncelikle veri dizinlerini tanımlıyoruz.
## - PyTorch'da genelde veri okume & ayrıştırma kısmı ve model oluşturma kısmı Pytorch class'larından inherit edilir.
## - Ben de bu bölümde "FruitImagesDataset" class'ını oluşturdum. Class input olarak veri dizini, görsel boyutu ve transform almaktadır. Class çalışma şekli:
### - Önce görsel okunur. BGR'den RGB'ye çevilir. Float cinsine çevrilir. Görsel istenen boyutlara indirgenir ve normalize edilir.
### - XML dosyası ayrıştırılır. Görsele ait Id, obje koordinatları, görsel sınıfı ve alanı hesaplanıp etikette tutulur.

In [ ]:
# Veriyolları
files_dir = '../input/fruit-images-for-object-detection/train_zip/train'
test_dir = '../input/fruit-images-for-object-detection/test_zip/test'


class FruitImagesDataset(torch.utils.data.Dataset):

    def __init__(self, files_dir, width, height, transforms=None):
        self.transforms = transforms
        self.files_dir = files_dir
        self.height = height
        self.width = width
        
        # JPG görüntülerini sıralama
        self.imgs = [image for image in sorted(os.listdir(files_dir))
                        if image[-4:]=='.jpg']
        
        
        # Sınıf isimleri(ekstra olarak FasterRCNN kullanıdğım için background koydum)
        self.classes = [_, 'apple','banana','orange']
    
    def __len__(self):
        return len(self.imgs)
        
    def __getitem__(self, idx):

        img_name = self.imgs[idx]
        image_path = os.path.join(self.files_dir, img_name)

        # Görsel okuma vs    
        img = cv2.imread(image_path)
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB).astype(np.float32)
        img_res = cv2.resize(img_rgb, (self.width, self.height), cv2.INTER_AREA)
        # Normalizasyon önemli
        img_res /= 255.0
        
        # anEtiket dosyası
        annot_filename = img_name[:-4] + '.xml'
        annot_file_path = os.path.join(self.files_dir, annot_filename)
        
        boxes = []
        labels = []
        tree = et.parse(annot_file_path)
        root = tree.getroot()
        
        # cgörsel boyutları
        wt = img.shape[1]
        ht = img.shape[0]
        
        # bbox koordinatları
        for member in root.findall('object'):
            labels.append(self.classes.index(member.find('name').text))
            
            # bounding box
            xmin = int(member.find('bndbox').find('xmin').text)
            xmax = int(member.find('bndbox').find('xmax').text)
            
            ymin = int(member.find('bndbox').find('ymin').text)
            ymax = int(member.find('bndbox').find('ymax').text)
            
            
            xmin_corr = (xmin/wt)*self.width
            xmax_corr = (xmax/wt)*self.width
            ymin_corr = (ymin/ht)*self.height
            ymax_corr = (ymax/ht)*self.height
            
            boxes.append([xmin_corr, ymin_corr, xmax_corr, ymax_corr])
        
        # Koordinatları Torch Tensor yapısına çevrilir
        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        
        # Bbox alanı
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])

        iscrowd = torch.zeros((boxes.shape[0],), dtype=torch.int64)
        
        labels = torch.as_tensor(labels, dtype=torch.int64)


        target = {}
        target["boxes"] = boxes
        target["labels"] = labels
        target["area"] = area
        target["iscrowd"] = iscrowd
        # Görsel Id
        image_id = torch.tensor([idx])
        target["image_id"] = image_id


        if self.transforms:
            
            sample = self.transforms(image = img_res,
                                     bboxes = target['boxes'],
                                     labels = labels)
            
            img_res = sample['image']
            target['boxes'] = torch.Tensor(sample['bboxes'])
            
            
            
        return img_res, target


# Veriyi okuyalım bakalım
dataset = FruitImagesDataset(files_dir, 224, 224)
print('length of dataset = ', len(dataset), '\n')

# Rastgele bir örnek alalım
img, target = dataset[78]
print(img.shape, '\n',target)

# Küçük bir görselleştirme..

In [ ]:
def plot_img_bbox(img, target):
    fig, a = plt.subplots(1,1)
    fig.set_size_inches(5,5)
    a.imshow(img)
    ## Birden fazla box obje bulunabilir.
    for box in (target['boxes']):
        x, y, width, height  = box[0], box[1], box[2]-box[0], box[3]-box[1]
        rect = patches.Rectangle((x, y),
                                 width, height,
                                 linewidth = 2,
                                 edgecolor = 'r',
                                 facecolor = 'none')

        a.add_patch(rect)
    plt.show()
    
# Rastgele bir görsele ve etiketine bakalım
img, target = dataset[25]
plot_img_bbox(img, target)

# Model - FasterRCNN
## - Popüler klasik modellerden bir tanesidir.
## - Faster R-CNN iki aşamada incelenebilir: 
### **Region Proposal Network (RPN):** İlk aşama olan RPN bölge önermeye yarayan derin, evrişimli bir sinir ağıdır. RPN, girdi olarak herhangi bir boyutta girdiyi alır ve obje skoruna göre bir dizi nesnelere ait olabilecek dikdörtgen teklifi ortaya çıkarır. Bu öneriyi, evrişimli katman tarafından oluşturulan öznitelik haritası üzerinde küçük bir ağı kaydırarak yapar.
### **Fast R-CNN:** RPN tarafından üretilen bu hesaplamalar Fast R-CNN mimarisine sokulur ve bir sınıflandırıcı ile objenin sınıfı, regressor ile de bounding box’u tahmin edilir.
![MİMARİ YAPI](https://d9v7j6n3.rocketcdn.me/wp-content/uploads/2020/12/faster-r-cnn.jpg.webp)

## [PyTorch FasterRCNN](https://pytorch.org/tutorials/intermediate/torchvision_tutorial.html) [Fast RCNN](https://teknoloji.org/nesne-tanima-algoritmalari-r-cnn-fast-r-cnn-ve-faster-r-cnn-nedir/)

In [ ]:

def get_object_detection_model(num_classes):

    # Daha önce eğitilmiş bir fasterrcnn modeli çekelim.
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
    
    # Input Features
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    # sınıf sayımız farklı olabileceğinden dolayı katmanları değiştiriyoruz
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes) 

    return model

# Veri Çoğaltma
# - Verisetinde toplamda 300 adet görsel bulunmaktadır. Aslında bu rakam çok fazla değildir. Data Augmentation veri sayısını arttırmak için kullanılır.
# - Bu sefer yapmadım fakat bazen veriye bulanıklık, parlaklık gibi gürültü eklemek modelimizin daha gürbüz(robust) çalışmasını sağlamaktadır.

In [ ]:
def get_transform(train):
    
    if train:
        return A.Compose([
                            A.HorizontalFlip(0.5),
                     # Dikey Flip işlemi
                            ToTensorV2(p=1.0) 
                        ], bbox_params={'format': 'pascal_voc', 'label_fields': ['labels']})
    else:
        return A.Compose([
                            ToTensorV2(p=1.0)
                        ], bbox_params={'format': 'pascal_voc', 'label_fields': ['labels']})

# Preparing dataset

In [ ]:
# Eğitim ve Test Verisini Okuyalım
dataset = FruitImagesDataset(files_dir, 480, 480, transforms= get_transform(train=True))
dataset_test = FruitImagesDataset(files_dir, 480, 480, transforms= get_transform(train=False))
# SEED
torch.manual_seed(1)
indices = torch.randperm(len(dataset)).tolist()

# Veriyi Sub-Train-Val-Test olarak ayrıştıralım
test_split = 0.2
tsize = int(len(dataset)*test_split)
dataset = torch.utils.data.Subset(dataset, indices[:-tsize])
dataset_test = torch.utils.data.Subset(dataset_test, indices[-tsize:])

# Veri DataLoader formatına çevirilir.
data_loader = torch.utils.data.DataLoader(
    dataset, batch_size=10, shuffle=True, num_workers=4,
    collate_fn=utils.collate_fn)

data_loader_test = torch.utils.data.DataLoader(
    dataset_test, batch_size=10, shuffle=False, num_workers=4,
    collate_fn=utils.collate_fn)

# Eğitim Ayarları
## - Öncelikle GPU cihazı seçilir.
## - Model çağırılır
## - Optimizer ve hiperparametre tanımlaması yapılır
## - İsteğe göre callbacks tanımalamaları yapılır. Ben bu denemede sadece "Learning Rate" düşürücü kullandım.

In [ ]:
# GPU SEÇME KISMI
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# SINIF SAYISI[ELMA,PORTAKAL,MUZ,ARKAPLAN]
# FasterRCNN Segmentation temelli bir yapı olduğundan "arkaplan" sınıfını ekliyoruz
num_classes = 4

# Model Çağırılır
model = get_object_detection_model(num_classes)

# Model GPU üzerine oturtulur
model.to(device)

# Optimizer ve Hiperparametre Tanımlaması.SGD seçilmiştir.
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005,
                            momentum=0.9, weight_decay=0.0005)

# Learning Rate Düşürücü
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                               step_size=3,
                                               gamma=0.1)

# Eğitim
## - 10 Epochluk bir eğitim tasarlanmıştır.
## - Hiperparamtereler, loss değerleri ve AP değerleri gözlenmektedir. 
## - Eğitim sonucunda yaklaşık test seti için 0.78-0.8 AP değerine ulaşılmıştır.

In [ ]:
# training for 10 epochs
num_epochs = 10

for epoch in range(num_epochs):
    # training for one epoch
    train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq=10)
    # update the learning rate
    lr_scheduler.step()
    # evaluate on the test dataset
    evaluate(model, data_loader_test, device=device)

# Modelimizi Test Edelim
## - Bu aşamada test setimizden rastgele bir örnek alıp çıktısını alalım.
## - Görmüş olduğunuz gibi model çıktısı 14 BBox gösterirken olması gereken BBox sayısı 1.
## - Bu farkın sebebi FasterRCNN NonMax Supression yapmamasıdır. "apply_nms" fonksiyonu model çıktısına NonMaxSupression yaparak en doğru bbox'u bulmaktadır.

In [ ]:
def apply_nms(orig_prediction, iou_thresh=0.3):
    
    # NonMax Supression Processing
    keep = torchvision.ops.nms(orig_prediction['boxes'], orig_prediction['scores'], iou_thresh)
    
    final_prediction = orig_prediction
    final_prediction['boxes'] = final_prediction['boxes'][keep]
    final_prediction['scores'] = final_prediction['scores'][keep]
    final_prediction['labels'] = final_prediction['labels'][keep]
    
    return final_prediction

# Torch2PILLOW
def torch_to_pil(img):
    return torchtrans.ToPILImage()(img).convert('RGB')

# rastgele bir veri seçelim
img, target = dataset_test[5]
# put the model in evaluation mode
model.eval()
with torch.no_grad():
    prediction = model([img.to(device)])[0]
    
print('predicted #boxes: ', len(prediction['labels']))
print('real #boxes: ', len(target['labels']))

## - Etiket Görselleştirmesi(Tek BBOX)

In [ ]:
print('EXPECTED OUTPUT')
plot_img_bbox(torch_to_pil(img), target)

## - Model Çıktısı (14 BBOX)

In [ ]:
print('MODEL OUTPUT')
plot_img_bbox(torch_to_pil(img), prediction)

## - Model çıktısını NonMaxSupression işleminden geçirince istediğimiz sonuca ulaştığını gözlemliyoruz.
## - Modelimizin tek dilimi de algıladığını gözlemliyoruz. Bu aslında istemediğimiz durumdur.
## - Bu durumdan kurtulmak için modelimizin başarımını arttırabilir, NonMaxSupression işlemiyle alakalı küçük değişikler yapabilir, veriye gürültü ekleyebiliriz.

In [ ]:
nms_prediction = apply_nms(prediction, iou_thresh=0.2)
print('NMS APPLIED MODEL OUTPUT')
plot_img_bbox(torch_to_pil(img), nms_prediction)

## Tüm test işlemleri tek bir kod parçacığında gösterelim. 
### - Rastgele bir görsel alıp, modelimizin nasıl sonuç verdiğine bakalım.
### - Modelimizin çıktısını NMS işleminden geçirelim, etiktele beraber görselleştirelim.

In [ ]:
test_dataset = FruitImagesDataset(test_dir, 480, 480, transforms= get_transform(train=True))
# Rastgele Görsel ve Etiket
img, target = test_dataset[10]
# EVAL MOD
model.eval()
with torch.no_grad():
    prediction = model([img.to(device)])[0]
    
print('EXPECTED OUTPUT\n')
plot_img_bbox(torch_to_pil(img), target)
print('MODEL OUTPUT\n')
nms_prediction = apply_nms(prediction, iou_thresh=0.01)

plot_img_bbox(torch_to_pil(img), nms_prediction)